In [50]:
%run dados.ipynb

In [51]:
spark = (SparkSession.builder
        .master("local")
        .appName("projetosoulcode")
        .config('spark.jars','gcs-connector-hadoop3-latest.jar' )
        .config('spark.hadoop.fs.gs.impl','com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem')
        .config('spark.hadoop.fs.gs.auth.service.account.enable', 'true')
        .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile",'soulcodeproject-44ce61622773.json')
        .config('spark.hadoop.fs.gs.project.id','soulcodeproject')
        .getOrCreate())
spark.conf.set("spark.sql.caseSensitive", "true")

In [52]:
schema = (
    StructType([
        StructField("ID",IntegerType(), False),
        StructField("AnoNascimento", IntegerType(), False),
        StructField("GrauInstrução", StringType(), True),
        StructField("EstadoCivil", StringType(), True),
        StructField("Renda", DoubleType(), True),
        StructField("NumCrianca", IntegerType(), True),
        StructField("NumAdolescente", IntegerType(), True),
        StructField("DataConta", StringType(), True),
        StructField("UltimaCompra", IntegerType(), True),
        StructField("QtdVinho", IntegerType(), True),
        StructField("QtdFrutas", IntegerType(), True),
        StructField("QtdCarne", IntegerType(), True),
        StructField("QtdPeixe", IntegerType(), True),
        StructField("QtdDoces", IntegerType(), True),
        StructField("QtdOuro", IntegerType(), True),
        StructField("QtdComprasDesconto", IntegerType(), True),
        StructField("QtdComprasWeb", IntegerType(), True),
        StructField("QtdCompraCatalogo", IntegerType(), True),
        StructField("QtdCompraLoja", IntegerType(), True),
        StructField("NumVisitasSite", IntegerType(), True),
        StructField("AceitouCam3", IntegerType(), True),
        StructField("AceitouCam4", IntegerType(), True),
        StructField("AceitouCam5", IntegerType(), True),
        StructField("AceitouCam1", IntegerType(), True),
        StructField("AceitouCam2", IntegerType(), True),
        StructField("Reclamacao", IntegerType(), True),
        StructField("CustoContato", IntegerType(), True),
        StructField("Receita", IntegerType(), True),
        StructField("UltimaCampanha", IntegerType(), True),
    ])
)

In [53]:
df2= (
    spark
       .read
       .format("csv")
       .option("header", "true")
       .option("delimiter", ",")
       .load('gs://soulcodeprojeto/marketing_campaign.csv',schema=schema)
)

In [54]:
df2.show()
print(df2.count(), len(df2.columns))

+----+-------------+-------------+-----------+-------+----------+--------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+------------+-------+--------------+
|  ID|AnoNascimento|GrauInstrução|EstadoCivil|  Renda|NumCrianca|NumAdolescente| DataConta|UltimaCompra|QtdVinho|QtdFrutas|QtdCarne|QtdPeixe|QtdDoces|QtdOuro|QtdComprasDesconto|QtdComprasWeb|QtdCompraCatalogo|QtdCompraLoja|NumVisitasSite|AceitouCam3|AceitouCam4|AceitouCam5|AceitouCam1|AceitouCam2|Reclamacao|CustoContato|Receita|UltimaCampanha|
+----+-------------+-------------+-----------+-------+----------+--------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------

In [55]:
a=[x[0] for x in df2.select('GrauInstrução').distinct().collect()]
df2=df2.replace(a, ['Segundo Ciclo', 'PhD', 'Mestrado', 'Graduado', 'Básico'])
df2.show()

+----+-------------+-------------+-----------+-------+----------+--------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+------------+-------+--------------+
|  ID|AnoNascimento|GrauInstrução|EstadoCivil|  Renda|NumCrianca|NumAdolescente| DataConta|UltimaCompra|QtdVinho|QtdFrutas|QtdCarne|QtdPeixe|QtdDoces|QtdOuro|QtdComprasDesconto|QtdComprasWeb|QtdCompraCatalogo|QtdCompraLoja|NumVisitasSite|AceitouCam3|AceitouCam4|AceitouCam5|AceitouCam1|AceitouCam2|Reclamacao|CustoContato|Receita|UltimaCampanha|
+----+-------------+-------------+-----------+-------+----------+--------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------

In [56]:
b=[x[0] for x in df2.select('EstadoCivil').distinct().collect()]
df2=df2.replace(b,['Só se vive uma vez', 'Juntado', 'Casado', 'Absurdo', 'Viúvo','Divorciado', 'Sozinho', 'Solteiro'])
df2.show()

+----+-------------+-------------+-----------+-------+----------+--------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+------------+-------+--------------+
|  ID|AnoNascimento|GrauInstrução|EstadoCivil|  Renda|NumCrianca|NumAdolescente| DataConta|UltimaCompra|QtdVinho|QtdFrutas|QtdCarne|QtdPeixe|QtdDoces|QtdOuro|QtdComprasDesconto|QtdComprasWeb|QtdCompraCatalogo|QtdCompraLoja|NumVisitasSite|AceitouCam3|AceitouCam4|AceitouCam5|AceitouCam1|AceitouCam2|Reclamacao|CustoContato|Receita|UltimaCampanha|
+----+-------------+-------------+-----------+-------+----------+--------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------

In [57]:
df2.select([count(when(isnan(c), c)).alias(c) for c in df2.columns]).show()

+---+-------------+-------------+-----------+-----+----------+--------------+---------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+------------+-------+--------------+
| ID|AnoNascimento|GrauInstrução|EstadoCivil|Renda|NumCrianca|NumAdolescente|DataConta|UltimaCompra|QtdVinho|QtdFrutas|QtdCarne|QtdPeixe|QtdDoces|QtdOuro|QtdComprasDesconto|QtdComprasWeb|QtdCompraCatalogo|QtdCompraLoja|NumVisitasSite|AceitouCam3|AceitouCam4|AceitouCam5|AceitouCam1|AceitouCam2|Reclamacao|CustoContato|Receita|UltimaCampanha|
+---+-------------+-------------+-----------+-----+----------+--------------+---------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+-----------

In [58]:
df2.select([count(when(col(c).isNull(), c)).alias(c) for c in df2.columns]).show()

+---+-------------+-------------+-----------+-----+----------+--------------+---------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+------------+-------+--------------+
| ID|AnoNascimento|GrauInstrução|EstadoCivil|Renda|NumCrianca|NumAdolescente|DataConta|UltimaCompra|QtdVinho|QtdFrutas|QtdCarne|QtdPeixe|QtdDoces|QtdOuro|QtdComprasDesconto|QtdComprasWeb|QtdCompraCatalogo|QtdCompraLoja|NumVisitasSite|AceitouCam3|AceitouCam4|AceitouCam5|AceitouCam1|AceitouCam2|Reclamacao|CustoContato|Receita|UltimaCampanha|
+---+-------------+-------------+-----------+-----+----------+--------------+---------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+-----------

In [59]:
df2=df2.dropna()

In [60]:
df2.describe().show()

+-------+----------------+------------------+-------------+-----------+------------------+------------------+------------------+----------+-----------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+-----------------+-----------------+-------------------+------------------+-------------------+-------------------+--------------------+-------------------+------------+-------+-------------------+
|summary|              ID|     AnoNascimento|GrauInstrução|EstadoCivil|             Renda|        NumCrianca|    NumAdolescente| DataConta|     UltimaCompra|          QtdVinho|         QtdFrutas|          QtdCarne|         QtdPeixe|          QtdDoces|          QtdOuro|QtdComprasDesconto|     QtdComprasWeb| QtdCompraCatalogo|    QtdCompraLoja|   NumVisitasSite|        AceitouCam3|       AceitouCam4|        AceitouCam5|        AceitouCam1|         AceitouCam2|         Recla

In [61]:
df2=df2.drop('Receita', 'CustoContato')

In [62]:
df2.summary().show()

+-------+----------------+------------------+-------------+-----------+------------------+------------------+------------------+----------+-----------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+-----------------+-----------------+-------------------+------------------+-------------------+-------------------+--------------------+-------------------+-------------------+
|summary|              ID|     AnoNascimento|GrauInstrução|EstadoCivil|             Renda|        NumCrianca|    NumAdolescente| DataConta|     UltimaCompra|          QtdVinho|         QtdFrutas|          QtdCarne|         QtdPeixe|          QtdDoces|          QtdOuro|QtdComprasDesconto|     QtdComprasWeb| QtdCompraCatalogo|    QtdCompraLoja|   NumVisitasSite|        AceitouCam3|       AceitouCam4|        AceitouCam5|        AceitouCam1|         AceitouCam2|         Reclamacao|     UltimaCamp

In [66]:
print(df2.count(), len(df2.columns))

2216 27
+----+-------------+-------------+-----------+-------+----------+--------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+--------------+
|  ID|AnoNascimento|GrauInstrução|EstadoCivil|  Renda|NumCrianca|NumAdolescente| DataConta|UltimaCompra|QtdVinho|QtdFrutas|QtdCarne|QtdPeixe|QtdDoces|QtdOuro|QtdComprasDesconto|QtdComprasWeb|QtdCompraCatalogo|QtdCompraLoja|NumVisitasSite|AceitouCam3|AceitouCam4|AceitouCam5|AceitouCam1|AceitouCam2|Reclamacao|UltimaCampanha|
+----+-------------+-------------+-----------+-------+----------+--------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+--------------+
|5524|         19

In [69]:
df2=df2.withColumnRenamed('NumCrianca','QtdCriancas').withColumnRenamed('NumAdolescente', 'QtdAdolescentes')
df2.show()

+----+-------------+-------------+-----------+-------+-----------+---------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+--------------+
|  ID|AnoNascimento|GrauInstrução|EstadoCivil|  Renda|QtdCriancas|QtdAdolescentes| DataConta|UltimaCompra|QtdVinho|QtdFrutas|QtdCarne|QtdPeixe|QtdDoces|QtdOuro|QtdComprasDesconto|QtdComprasWeb|QtdCompraCatalogo|QtdCompraLoja|NumVisitasSite|AceitouCam3|AceitouCam4|AceitouCam5|AceitouCam1|AceitouCam2|Reclamacao|UltimaCampanha|
+----+-------------+-------------+-----------+-------+-----------+---------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+--------------+
|5524|         1957

In [ ]:
df2.groupBy("AnoNascimento").agg({'QtdComprasWeb':'sum','QtdCompraCatalogo':'sum' }).show()